In [1]:
%use kotlin-statistics, krangl, kravis, klaxon

In [2]:
import kotlin.math.*

In [3]:
@file:Repository("https://dl.bintray.com/kyonifer/maven")
@file:DependsOn("com.kyonifer:koma-core-ejml:0.12")
@file:DependsOn("com.kyonifer:koma-plotting:0.12")

In [4]:
// @file:Repository("https://repo1.maven.org/maven2")
@file:DependsOn("com.github.haifengl:smile-core:2.2.2")

In [5]:
import smile.clustering.*
import smile.neighbor.*

In [6]:
import smile.*

In [7]:
import koma.extensions.*
import koma.*

In [8]:
val df = DataFrame.readTSV("UI_1-2buckets_6march_filt.tsv")

In [9]:
val df2 = DataFrame.readTSV("UI_1-2buckets_6march.tsv")

In [10]:
class Action(val time: Long, val event_id: String, val group_id: String, val dt: String) {
    fun print() {
        println("$time $event_id $group_id $dt")
    }
    
    fun get_event(): String {
        return group_id + "_" + dt
    }
}

In [11]:
class Session(val id: String) {
    public var actions: ArrayList<Action> = ArrayList()

    fun add(time: Long, event: String, group: String, action: String) {
        actions.add(Action(time, event, group, action))
    }

    fun print() {
        println(id)
        for (action in actions) {
            action.print()
        }
    }
    
    fun time(): Long {
        val t: Long = actions[actions.size-1].time
        return t / 100 / 60
    }
    
    fun events(): String {
        val events = ArrayList<String>()
        for (action in actions)
            events.add(action.get_event())
        return events.joinToString(separator = " , ")
    }
}

In [18]:
var vc = ArrayList<Long>()

In [20]:
fun makeSessions(path: String, threshold: Long = 72000): ArrayList<Session> {
    val df = DataFrame.readTSV(path)
    val groups = df.groupBy("device_id").sortedBy("time_epoch").groups()
    val ses: ArrayList<Session> = ArrayList()
    var tot = -1
    for (group in groups) {
        var flag = false
        var cnt: Int = -1
        var prev: Long = -1
        var start: Long = -1
        // ses.add(Session(groups[0].row(0)['device_id']))
        for (row in group.rows) {
            
            
            
            val dev = row.getOrDefault("device_id", "") as String
            val time = row.getOrDefault("time_epoch", 0) as Long
            val event = row.getOrDefault("event_id", "") as String
            val groupId = row.getOrDefault("group_id", "") as String
            val dt = row.getOrDefault("data", "") as String
            
            if (prev != 1L && flag) {
                vc.add(time-prev)
                flag = false
            }
            if (event == "fetch.finished")
                flag = true
            
            
            if (groupId == "lifecycle")
                if (event == "frame.activated" || event == "frame.deactivated")
                    continue
            if (prev == -1L || time - prev > threshold) {
                cnt++
                tot++
                start = time
                ses.add(Session(dev + "_" + cnt.toString()))
                ses[tot].add(time - start, event, groupId, dt)
            } else {
                ses[tot].add(time - start, event, groupId, dt)
            }
            prev = time

        }
    }
    return ses
}


In [21]:
val ses = makeSessions("UI_1-2buckets_6march_filt.tsv")

In [28]:
1.0 * vc.sum() / vc.size / 100 / 60

47.14557313681869

In [17]:
for (s in ses){
    if (s.actions.size == 3)
        s.print()

}

25101933e9a2c4e-9cdc-4acc-9688-e7b99555246d_7
0 feature.used productivity editing.select.word
11691 BreadcrumbShowTooltip ui.event BreadcrumbShowTooltip
13057 activated toolwindow unknown
0505191d5ca0393-2e1d-4356-b3d2-afba2ab75a37_4
0 edit file.types.usage JAVA
1025 action.invoked actions EditorBackSpace
9290 activated toolwindow Run
26022018d46b541-c82a-45c3-9328-9aff8a5579c7_20
0 BreadcrumbShowTooltip ui.event BreadcrumbShowTooltip
3192 feature.used productivity editing.select.word
5537 BreadcrumbShowTooltip ui.event BreadcrumbShowTooltip
26022018d46b541-c82a-45c3-9328-9aff8a5579c7_23
0 BreadcrumbShowTooltip ui.event BreadcrumbShowTooltip
2112 feature.used productivity editing.select.word
24390 feature.used productivity editing.select.word
04022010fe9e84f-f727-4b03-9638-947668f701ca_2
0 action.invoked actions editRunConfigurations
80 show ui.dialogs com.intellij.execution.impl.EditConfigurationsDialog
2360 close ui.dialogs com.intellij.execution.impl.EditConfigurationsDialog
1505192

In [14]:
fun dist(s1: Session, s2: Session): Double {
    val a1 = ArrayList<String>()
    val a2 = ArrayList<String>()

    for (a in s1.actions) {
        a1.add(a.event_id + " " + a.group_id + " " + a.dt)
    }

    for (a in s2.actions) {
        a2.add(a.event_id + " " + a.group_id + " " + a.dt)
    }

    val h1 = a1.groupingBy { it }.eachCount()
    val h2 = a2.groupingBy { it }.eachCount()

    val keys = HashSet<String>()
    keys.addAll(h1.keys)
    keys.addAll(h2.keys)

    val h3 = HashMap<String, Int>()
    for (k in keys) {
        h3[k] = min(h1.getOrDefault(k, 0), h2.getOrDefault(k, 0))
    }

    val v1 = h1.values.sum()
    val v2 = h2.values.sum()
    val v3 = h3.values.sum()

    val f1: Double = (1.0 * v3) / v1
    val f2: Double = (1.0 * v3) / v2

    if (f1 + f2 == 0.0)
        return 0.0
    return 2 * f1 * f2 / (f1 + f2)

}

In [16]:
dist(ses[0], ses[1])

0.18518518518518515

In [18]:
dist2(ses[0].events(), ses[1].events())

0.18518518518518515

In [13]:
class DistSes<T>: smile.math.distance.Distance<T> {
    /**
     * Returns the distance measure between two objects.
     */
    override fun d(x: T, y: T): Double {
        if (x is Session && y is Session)
            return 1 - dist(x, y)
        return 1.0
    }

}

In [16]:
ses.shuffle()
val res: ArrayList<Session> = ses.slice(0 .. 1000) as ArrayList<Session>

In [26]:
val s: Array<Session> = Array<Session>(1001) {ses[0]}

In [33]:
for (i in 0 until 1001) {
    s[i] = res[i]
}

In [17]:
fun dist2(s1: String, s2: String): Double {
    val a1 = s1.split(" , ")
    val a2 = s2.split(" , ")
    
    val h1 = a1.groupingBy { it }.eachCount()
    val h2 = a2.groupingBy { it }.eachCount()

    val keys = HashSet<String>()
    keys.addAll(h1.keys)
    keys.addAll(h2.keys)

    val h3 = HashMap<String, Int>()
    for (k in keys) {
        h3[k] = min(h1.getOrDefault(k, 0), h2.getOrDefault(k, 0))
    }

    val v1 = h1.values.sum()
    val v2 = h2.values.sum()
    val v3 = h3.values.sum()

    val f1: Double = (1.0 * v3) / v1
    val f2: Double = (1.0 * v3) / v2

    if (f1 + f2 == 0.0)
        return 0.0
    return 2 * f1 * f2 / (f1 + f2)

}

In [ ]:
ses[0]

In [17]:
var clusters = smile.clustering.DBSCAN.fit(res.toArray(), DistSes(), 10, 0.5)

java.lang.NoSuchMethodError: Line_16_jupyter$DistSes: method <init>()V not found
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:95)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:40)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvmhost.repl.JvmReplEvaluator$eval$$inlined$write$lambda$2.invokeSuspend(legacyReplEvaluation.kt:57)
kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationImp

In [115]:
var clusters = smile.clustering.DBSCAN.fit(ses.toArray(), DistSes(), 10, 0.5)

java.lang.NoSuchMethodError: Line_105_jupyter$DistSes: method <init>()V not found
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.evalWithConfigAndOtherScriptsResults(BasicJvmScriptEvaluator.kt:95)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke$suspendImpl(BasicJvmScriptEvaluator.kt:40)
kotlin.script.experimental.jvm.BasicJvmScriptEvaluator.invoke(BasicJvmScriptEvaluator.kt)
kotlin.script.experimental.jvmhost.repl.JvmReplEvaluator$eval$$inlined$write$lambda$2.invokeSuspend(legacyReplEvaluation.kt:57)
kotlin.coroutines.jvm.internal.BaseContinuationImpl.resumeWith(ContinuationIm

In [186]:
var d = ArrayList<String>()
for (s in ses){
    for (a in s.actions) {
        if (a.group_id == "build.maven.packagesearch") {
            d.add(a.event_id)
        }
    }
}

In [190]:
d[1]

{"endpoint":"fulltext","os":"Mac","created":1583470706817,"time":1248,"timeout":2000}

In [191]:
ses

[Line_11_jupyter$Session@6b828892, Line_11_jupyter$Session@72aa9fd2, Line_11_jupyter$Session@1d118f0b, Line_11_jupyter$Session@6e5f0099, Line_11_jupyter$Session@1166d3d8, Line_11_jupyter$Session@37949a29, Line_11_jupyter$Session@266c2207, Line_11_jupyter$Session@30a1a59b, Line_11_jupyter$Session@17fe4844, Line_11_jupyter$Session@428cd40, Line_11_jupyter$Session@4393abd8, Line_11_jupyter$Session@7e23434c, Line_11_jupyter$Session@1bfa88a4, Line_11_jupyter$Session@59fa1456, Line_11_jupyter$Session@769a9da, Line_11_jupyter$Session@63b4a358, Line_11_jupyter$Session@2c44b822, Line_11_jupyter$Session@25154158, Line_11_jupyter$Session@6ff81b59, Line_11_jupyter$Session@2990cb9d, Line_11_jupyter$Session@201ce325, Line_11_jupyter$Session@2f8c79ff, Line_11_jupyter$Session@819d84, Line_11_jupyter$Session@2fe2622a, Line_11_jupyter$Session@6820a938, Line_11_jupyter$Session@24af7642, Line_11_jupyter$Session@284ea2ee, Line_11_jupyter$Session@148176ff, Line_11_jupyter$Session@7905643e, Line_11_jupyter$S

In [13]:
idclose.size

618

In [86]:
idclose.sum() / 100 / 60 / idclose.size

63

In [14]:
val lifecycle = ArrayList<Int>()

In [19]:
var ind = 0
for (sess in ses) {
    lifecycle.add(0)
    for (action in sess.actions) {
        if (action.group_id == "lifecycle") {
            if (action.event_id == "frame.activated")
                lifecycle[ind]++   
            if (action.event_id == "frame.deactivated")
                lifecycle[ind]--
            if (lifecycle[ind] < 0)
                println("no")
                break
        } 
    }
    if (lifecycle[ind] < 0)
        break
    ind ++
}

no


In [34]:
var small = ArrayList<Session>()
for (sess in ses){
    if (sess.actions.size < 10)
        small.add(sess)
}

In [36]:
ses.size

8302

In [95]:
small[10].print()

0705193eb2dae1e-9342-408a-8ab1-bac507bad3eb_22
0 fetch.started vcs 
44 fetch.finished vcs 


In [30]:
ses[2].print()

2204192da927d8d-1396-4efd-8e1a-18b59ebf81c6_2
0 whitelist.loaded event.log 
1938 ide.start lifecycle 
3835 show ui.dialogs 
11495 project.opened lifecycle 
24272 dialog.shown ui.tips 
24396 tip.shown ui.tips 
24482 show ui.dialogs 
27190 activated toolwindow 
27540 close ui.dialogs 
28199 project.opening.finished lifecycle 
33669 splash startup 
33669 bootstrap startup 
33669 appInit startup 
33669 totalDuration startup 
102181 close ui.dialogs 
104753 NavBarShowPopup ui.event 
108954 action.invoked actions ActivateProjectToolWindow
108963 activated toolwindow 
126528 open file.types.usage 
135466 open file.types.usage 
158461 action.invoked actions com.intellij.codeInsight.daemon.impl.DaemonTooltipWithActionRenderer$SettingsActionGroup
158576 clicked toolbar com.intellij.codeInsight.daemon.impl.DaemonTooltipWithActionRenderer$SettingsActionGroup
161585 action.invoked actions com.intellij.codeInsight.daemon.impl.DaemonTooltipWithActionRenderer$ShowDocAction
161588 show.description tool

In [31]:
ses[4].print()

25101933e9a2c4e-9cdc-4acc-9688-e7b99555246d_4
0 whitelist.loaded event.log 
4420 ide.start lifecycle 
6876 action.invoked actions WelcomeScreen.ImportProject
7607 show ui.dialogs 
40889 close ui.dialogs 
43137 action.invoked actions com.intellij.openapi.wm.impl.welcomeScreen.FlatWelcomeFrame$FlatWelcomeScreen$5
45253 action.invoked actions WelcomeScreen.Plugins
45413 show ui.dialogs 
48256 action.invoked actions com.intellij.ide.plugins.newui.TabHeaderComponent$2
48294 clicked toolbar com.intellij.ide.plugins.newui.TabHeaderComponent$2
50252 action.invoked actions com.intellij.ide.plugins.PluginManagerConfigurable$3
50301 show ui.dialogs 
52992 action.invoked actions com.intellij.ui.CommonActionsPanel$MyActionButton
52997 clicked toolbar com.intellij.ui.CommonActionsPanel$MyActionButton
173034 action.invoked actions EditorPaste
174762 action.invoked actions EditorBackSpace
176066 apply ui.settings 
176066 close ui.dialogs 
176320 show ui.dialogs 
177767 close ui.dialogs 
177829 show ui

In [201]:
lifecycle[5]

[1, -1]

In [192]:
lifecycle[0].print()

1312191092f7f25-dafb-4872-bdef-48d862040e85_0
0 frame.activated lifecycle 
7643 action.invoked actions CheckinProject
7982 show ui.dialogs 
8083 show ui.dialogs 
8121 close ui.dialogs 
8669 edit file.types.usage 
8683 finished completion 
8908 finished completion 
8978 finished completion 
9158 finished completion 
9267 finished completion 
9543 action.invoked actions EditorBackSpace
10225 action.invoked actions $SelectAll
10485 finished completion 
10872 finished completion 
10958 finished completion 
11103 finished completion 
11604 action.invoked actions EditorBackSpace
13554 finished completion 
13784 finished completion 
13931 finished completion 
14087 finished completion 
14411 finished completion 
14513 finished completion 
14642 finished completion 
14883 finished completion 
15099 finished completion 
15209 finished completion 
15708 finished completion 
16774 finished completion 
16898 finished completion 
17080 finished completion 
17149 finished completion 
17244 finished 

In [119]:
events.size

69

In [112]:
for (event in events)
    println(event)

validation.unmatched_rule ui.event
find.method.started java.find.usages
validation.unreachable.whitelist file.editor
GradlePerformance kotlin.gradle.performance
check.box.toggled find
custom.action.invoked actions
fetch.started vcs
validation.unreachable.whitelist plugins.advertiser
validation.unreachable.whitelist event.log
dialogOpen searchEverywhere
validation.unreachable.whitelist file.types.usage
finished run.configuration.exec
validation.unreachable.whitelist searchEverywhere
frame.activated lifecycle
NavBarNavigate ui.event
validation.undefined_rule startup
BundledPluginGroupCustomized customize.wizard
Completion kotlin.ide.editor
ui.shown run.configuration.exec
Kotlin_Object kotlin.ide.newFileTempl
feature.used productivity
expanded completion.postfix
attempt.add.framework new.project.wizard
validation.unreachable.whitelist js.language.service
validation.unmatched_rule ui.tips
Kotlin_File kotlin.ide.newFileTempl
SuppressInspectionUsed shell.script
ShowUsagesPopup.showSettings t

In [109]:
events.size

216

In [94]:
ses[1].print()

1312191092f7f25-dafb-4872-bdef-48d862040e85_1
0 frame.activated lifecycle 
4535 action.invoked actions Git.Tag
4665 show ui.dialogs 
6627 action.invoked actions EditorBackSpace
8497 frame.deactivated lifecycle 
9596 frame.activated lifecycle 
9934 action.invoked actions EditorBackSpace
11628 close ui.dialogs 
11628 custom.action.invoked actions DialogOkAction
17164 action.invoked actions Vcs.Push
17179 show ui.dialogs 
18678 close ui.dialogs 
22528 ide.close lifecycle 
22683 project.closed lifecycle 


In [73]:
ses[0].actions[0].time - ses[0].actions[0].time 

1583453289154

In [77]:
var res = HashMap<Pair<Int, Int>, HashMap<String, Double>>()

In [84]:
for (i in 1 until 11) {
    for (j in 1 until 11) {
        val df = DataFrame.readTSV("test/test_$i _$j.tsv")
        val gr = df.filterByRow {(it["clust"] as Int) < 1000}.groupBy("clust").groups()
        var vl = HashMap<String, Double>()
        for (g in gr) {
            val ht = HashMap<String, Int>()
            for (i in 0 until g.nrow) {
                val cats = g["category"][i].toString().split(", ")
                for (cat in cats) {
                    if (ht.contains(cat)) {
                        ht[cat] = ht.getOrDefault(cat, 0) + 1
                    }
                    else {
                        ht[cat] = 0
                    }
                }
            }
            var k:String = "lol"
            for (e in ht.keys) {
                val pm:Int = ht.getOrDefault(k, 0)
                val cv:Int = ht.getOrDefault(e, 0)
                if (cv > pm) {
                    k = e
                }
            }
            if (k != "lol")
                vl[k] = 1.0 * ht.getOrDefault(k, 0) / g.nrow
        }
        res[Pair(i, j)] = vl
    }
}

In [89]:
for (i in 1 until 11) 
    for (j in 1 until 11) {
        print("$i $j ")
        println(res!![Pair(i, j)])
    }

1 1 {ide start=0.6666666666666666, ide close=0.3333333333333333}
1 2 {ide start=0.5, ide close=0.5}
1 3 {coding=0.3333333333333333, ide start=0.5, ide close=0.5}
1 4 {coding=0.3333333333333333, ide start=0.5, ide close=0.7142857142857143}
1 5 {coding=0.5, RD=0.6666666666666666, ide start=0.5, ide close=0.8}
1 6 {coding=0.6666666666666666, ide start=0.5}
1 7 {coding=0.5, vcs=0.5, ide start=0.5}
1 8 {coding=0.543859649122807, ide start=0.5}
1 9 {coding=0.5344827586206896}
1 10 {coding=0.5371428571428571}
2 1 {ide start=0.6666666666666666, ide close=0.3333333333333333}
2 2 {ide start=0.8888888888888888, ide close=0.5}
2 3 {coding=0.3333333333333333, ide start=0.8571428571428571, ide close=0.5}
2 4 {coding=0.3333333333333333, ide start=0.8333333333333334, ide close=0.7142857142857143}
2 5 {coding=0.6, RD=0.6666666666666666, ide start=0.8108108108108109, ide close=0.8}
2 6 {coding=0.6666666666666666, ide start=0.47368421052631576}
2 7 {coding=0.5066666666666667}
2 8 {coding=0.54385964912280

In [52]:
val g = gr[0]

In [56]:
g["category"][0]

Ide start, Ide close

In [64]:
val ht = HashMap<String, Int>()
for (i in 0 until g.nrow) {
    val cats = g["category"][i].toString().split(", ")
    for (cat in cats) {
        if (ht.contains(cat)) {
            ht[cat] = ht.getOrDefault(cat, 0) + 1
        }
        else {
            ht[cat] = 0
        }
    }
}
var k:String = "lol"
for (e in ht.keys) {
    val pm:Int = ht.getOrDefault(k, 0)
    val cv:Int = ht.getOrDefault(e, 0)
    if (cv > pm) {
        k = e
    }
}


In [76]:
k

ide start

In [74]:
ht.keys

[coding, rd, RD, VCS, vcs, Ide start, terminal, Ide close, ide start, db, ide close]

In [63]:
g["category"][0]

Ide start, Ide close